# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from sql_queries import *
from functions import *

In [2]:
# checking current working directory
pwd = os.getcwd()

# calling preprocessing function
preprocessing_csv_file(pwd)

# check the number of rows in new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. ETL Pipeline for Creating and Querying NoSQL Database

### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="pics/event_datafile.png">

## A. Create Connection and Keyspace

#### Create a Cluster and Connection to Database

In [3]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [4]:
try:
    session.execute(create_keyspace)
except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## B. Create Denormalized Tables

### B1. Session Table
**Query Example:** <br> Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### CREATE Table

In [6]:
session.execute(session_table_drop)

try:
    session.execute(session_table_create)
except Exception as e:
    print(e)                  

#### INSERT into Table

In [7]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        session.execute(session_table_insert, 
                        (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement

In [9]:
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(session_table_select, (338, 4))
except Exception as e:
    print(e)

data = []
for row in rows:
    data.append(row)
    df = pd.DataFrame(data)
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### B2. User Table
**Query Example:** <br> Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

#### CREATE Table

In [10]:
session.execute(user_table_drop)

try:
    session.execute(user_table_create)
except Exception as e:
    print(e)

#### INSERT into Table

In [11]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(user_table_insert, 
                        (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))

#### SELECT statement

In [12]:
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(user_table_select, (10, 182))
except Exception as e:
    print(e)
    
data = []
for row in rows:
    data.append(row)
    df = pd.DataFrame(data)
df

,artist,song,iteminsession,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


### B3. Song Table
**Query Example:** <br> Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### CREATE Table

In [13]:
session.execute(song_table_drop)

try:
    session.execute(song_table_create)
except Exception as e:
    print(e)

#### INSERT into Table

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(song_table_insert, 
                        (line[1], line[4], int(line[10]), line[9]))

#### SELECT statement

In [15]:
# SELECT statement with specifications made in query example above
try:
    rows = session.execute(song_table_select, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
data = []
for row in rows:
    data.append(row)
    df = pd.DataFrame(data)
df

,firstname,lastname,userid,song
0,Jacqueline,Lynch,29,All Hands Against His Own
1,Tegan,Levine,80,All Hands Against His Own
2,Sara,Johnson,95,All Hands Against His Own


## C. Drop Tables, Close Session and Cluster Connection¶

In [16]:
session.execute(session_table_drop)
session.execute(user_table_drop)
session.execute(song_table_drop)

In [17]:
session.shutdown()
cluster.shutdown()